DATA PREPROCESSING

In [6]:
# DEFINITIONS AND IMPORTING

import pandas as pd
import numpy as np
import scipy.stats as stats
import sklearn.linear_model as lm
import sklearn.ensemble as se
import sklearn.metrics as sm
import sklearn.model_selection as sms
import sklearn.neighbors as sn
import xgboost as xgb
import matplotlib.pyplot as pyplt
import boruta as bt
import catboost as cb
import optuna
from IPython.display import clear_output
import warnings
import pickle
import shap
import os
warnings.simplefilter('ignore')
np.random.seed(42)

np.int = np.int64
np.float = np.float64
np.bool = np.bool_

SKF = sms.StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

clear_output()

In [7]:
# FILTER LOADING

stat_sel_icc = pd.read_pickle("databases/statistically_selected_features_icc.pkl")
stat_sel_pval1 = pd.read_pickle("databases/statistically_selected_features_pval1.pkl")
filt = [stat_sel_pval1, stat_sel_icc]

# DATA LOADING

derived_keys1 = ["native", "1_pc", "2_pc", "3_pc", "4_pc", "5_pc"]
derived_keys2 = ["native"]

def process_data(data_X, feature_filters, derived_keys):

    load_X = data_X

    df_derived = pd.concat([load_X[key] for key in derived_keys])
    load_X["feat_mean"] = df_derived.groupby(df_derived.index).mean()
    load_X["feat_max_diff"] = df_derived.groupby(df_derived.index).max() - df_derived.groupby(df_derived.index).min()
    load_X["feat_std"] = df_derived.groupby(df_derived.index).std()
    load_X["feat_max"] = df_derived.groupby(df_derived.index).max()
    load_X["feat_min"] = df_derived.groupby(df_derived.index).min()

    for i in load_X.keys():
        if(feature_filters!=None):
            temp_list = list(feature_filters[0][i])
            for j in feature_filters:
                temp_list = list(set(temp_list).intersection(list(j[i])))
            load_X[i] = load_X[i][temp_list]
    
    for i in load_X.keys():
        load_X[i].columns += f"_{i}"
    
    return_X = pd.concat(load_X.values(), axis=1)
    return return_X

db_1 = pd.read_pickle("databases/database_1_X.pkl")
for i in db_1.keys():
    db_1[i].reset_index(inplace=True)
    ind = db_1[i]["seg_name"]
    db_1[i].drop("seg_name", axis=1, inplace=True)

X_all = process_data(db_1, filt, derived_keys1)
X_native = process_data(db_1, None, derived_keys2)
X_native.dropna(axis=1, how="any", inplace=True)

db_y = pd.read_csv("databases/database_1_y.csv")
y_all = db_y.drop("seg_name", axis=1)

ML PIPELINE

In [8]:
def RandomForestClassifier_trial_data(trial):
  return {"n_estimators" : trial.suggest_int("n_estimators", 10, 4000, log=True),
            "max_depth" : trial.suggest_int("max_depth", 1, 20),
            "min_samples_split" : trial.suggest_int("min_samples_split", 2, 20),
            "min_samples_leaf" : trial.suggest_int("min_samples_leaf", 1, 20),
            "max_leaf_nodes" : trial.suggest_int("max_leaf_nodes", 2, 20),
            "min_impurity_decrease" : trial.suggest_float("min_impurity_decrease", 0, 0.1),
            "min_weight_fraction_leaf" : trial.suggest_float("min_weight_fraction_leaf", 0, 0.5),
            "random_state" : trial.suggest_categorical("random_state", [42]),
            "max_features" : trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
            "bootstrap" : trial.suggest_categorical("bootstrap", [True, False])}

def XGBoostClassifier_trial_data(trial):
  return {"booster" : "gbtree",
            "n_estimators" : trial.suggest_int("n_estimators", 10, 4000, log=True),
            "max_depth" : trial.suggest_int("max_depth", 1, 20),
            "max_delta_step" : trial.suggest_float("max_delta_step", 0, 10),
            "learning_rate" : trial.suggest_float("learning_rate", 0.05, 1, log=True),
            "alpha" : trial.suggest_float("alpha", 0.01, 100, log=True),
            "gamma" : trial.suggest_float("gamma", 0.01, 1, log=True),
            "lambda" : trial.suggest_float("lambda", 0.01, 100, log=True),
            "colsample_bytree" : trial.suggest_float("colsample_bytree", 0.1, 1),
            "colsample_bylevel" : trial.suggest_float("colsample_bylevel", 0.1, 1),
            "random_state" : trial.suggest_categorical("random_state", [42])}

def CatBoostClassifier_trial_data(trial):
  return {"iterations" : trial.suggest_int("iterations", 50, 500, log=True),
          "learning_rate" : trial.suggest_float("learning_rate", 0.005, 1, log=True),
           "depth" : trial.suggest_int("depth", 1, 10),
           "min_data_in_leaf" : trial.suggest_int("min_data_in_leaf", 1, 150),
           "random_state" : trial.suggest_categorical("random_state", [42]),
           "colsample_bylevel" : trial.suggest_float("colsample_bylevel", 0.1, 1)}

def RidgeLogistic_trial_data(trial):
    return {"penalty" : trial.suggest_categorical("penalty", ["l2"]),
            "random_state" : trial.suggest_categorical("random_state", [42]),
            "solver" : trial.suggest_categorical("solver", ["liblinear", "saga"]),
            "max_iter" : trial.suggest_int("max_iter", 100, 3000),
            "C" : trial.suggest_float("C", 0.01, 1)}

def optuna_trial_hyperparametrized(trial, model_name):
  match model_name:
    case "RandomForestClassifier":
      return se.RandomForestClassifier(**(RandomForestClassifier_trial_data(trial)))
    case "XGBoostClassifier":
      return xgb.XGBClassifier(**(XGBoostClassifier_trial_data(trial)))
    case "CatBoostClassifier":
      return cb.CatBoostClassifier(**(CatBoostClassifier_trial_data(trial)))
    case "RidgeLogistic":
      return lm.LogisticRegression(**(RidgeLogistic_trial_data(trial)))
  return None

def optuna_result_hyperparametrized(params, model_name):
  match model_name:
    case "RandomForestClassifier":
      return se.RandomForestClassifier(**params)
    case "XGBoostClassifier":
      return xgb.XGBClassifier(**params)
    case "CatBoostClassifier":
      return cb.CatBoostClassifier(**params)
    case "RidgeLogistic":
        return lm.LogisticRegression(**params)
  return None

def hyperparameter_optimization(X, y, model_name):
  def objective(trial):
    model = optuna_trial_hyperparametrized(trial, model_name)
    scores = sms.cross_validate(model, X, y, cv=SKF, scoring=("accuracy", "f1", "roc_auc"), n_jobs=-1)
    return (np.mean(scores["test_accuracy"]) + np.mean(scores["test_f1"]) + 5*np.mean(scores["test_roc_auc"]))        
  study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
  study.optimize(objective, n_trials=50, n_jobs=-1)
  return study.best_params

def feature_selection(X, y):
  hyperparameters = hyperparameter_optimization(X, y, "XGBoostClassifier")
  fs_estimator = optuna_result_hyperparametrized(hyperparameters, "XGBoostClassifier")
  fs_boruta = bt.BorutaPy(fs_estimator, n_estimators="auto", verbose=2, random_state=42, perc=80).fit(X.to_numpy(), y.to_numpy())
  return list(X.columns[fs_boruta.support_ | fs_boruta.support_weak_])

In [ ]:
def pipeline(X, y, model_name, ttype):

    train_X, test_X, train_y, test_y = sms.train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

    feature_file = f"features/selected_features_{ttype}.pkl"
    if(os.path.isfile(feature_file) == False):
        selected_features = feature_selection(train_X, train_y)
        output = open(feature_file, "wb")
        pickle.dump(selected_features, output)
        output.close()
    selected_features = pd.read_pickle(feature_file)
    print(len(selected_features))
    train_X = train_X[selected_features]
    test_X = test_X[selected_features]

    param_file = f"hyperparameters/hyperparameters_{model_name}_{ttype}.pkl"
    if(os.path.isfile(param_file) == False):
        hyperparameters = hyperparameter_optimization(train_X, train_y, model_name)
        output = open(param_file, "wb")
        pickle.dump(hyperparameters, output)
        output.close()
    hyperparameters = pd.read_pickle(param_file)
    model = optuna_result_hyperparametrized(hyperparameters, model_name)

    model.fit(train_X, train_y)
    pred_y = model.predict(test_X)
    prob_y = model.predict_proba(test_X)

    accuracy = round(sm.accuracy_score(test_y, pred_y), 2)
    f1 = round(sm.f1_score(test_y, pred_y), 2)
    roc_auc = round(sm.roc_auc_score(test_y, prob_y[:,1]), 2)

    clear_output()
    
    print(f"Accuracy = {accuracy}")
    print(f"F1 = {f1}")
    print(f"ROC_AUC = {roc_auc}")

    full_model = optuna_result_hyperparametrized(hyperparameters, model_name)
    full_model.fit(X, y)
    output = open(f"models/model_{model_name}_{ttype}.pkl", "wb")
    pickle.dump(full_model, output)
    output.close()

    return

MODELLING - ALL CONTRASTS

In [10]:
pipeline(X_all, y_all, "RandomForestClassifier", "all")

Accuracy = 0.66
F1 = 0.27
ROC_AUC = 0.74


In [11]:
pipeline(X_all, y_all, "XGBoostClassifier", "all")

Accuracy = 0.78
F1 = 0.7
ROC_AUC = 0.7


In [12]:
pipeline(X_all, y_all, "CatBoostClassifier", "all")

Accuracy = 0.69
F1 = 0.55
ROC_AUC = 0.72
0:	learn: 0.5988386	total: 8.22ms	remaining: 1.31s
1:	learn: 0.5265962	total: 16.8ms	remaining: 1.32s
2:	learn: 0.4664024	total: 30.7ms	remaining: 1.61s
3:	learn: 0.4278604	total: 76.3ms	remaining: 2.97s
4:	learn: 0.3841791	total: 93.1ms	remaining: 2.89s
5:	learn: 0.3130186	total: 103ms	remaining: 2.64s
6:	learn: 0.2582601	total: 110ms	remaining: 2.4s
7:	learn: 0.2183585	total: 119ms	remaining: 2.26s
8:	learn: 0.2029317	total: 127ms	remaining: 2.13s
9:	learn: 0.1769777	total: 138ms	remaining: 2.07s
10:	learn: 0.1330210	total: 146ms	remaining: 1.97s
11:	learn: 0.1105319	total: 153ms	remaining: 1.89s
12:	learn: 0.1052220	total: 161ms	remaining: 1.82s
13:	learn: 0.0761909	total: 169ms	remaining: 1.76s
14:	learn: 0.0728269	total: 177ms	remaining: 1.71s
15:	learn: 0.0624582	total: 185ms	remaining: 1.66s
16:	learn: 0.0527935	total: 193ms	remaining: 1.62s
17:	learn: 0.0444151	total: 200ms	remaining: 1.58s
18:	learn: 0.0430712	total: 219ms	remaining: 1.

In [13]:
pipeline(X_all, y_all, "RidgeLogistic", "all")

Accuracy = 0.69
F1 = 0.38
ROC_AUC = 0.62


MODELING - NATIVE CONTRAST

In [14]:
pipeline(X_native, y_all, "RandomForestClassifier", "native")

Accuracy = 0.72
F1 = 0.47
ROC_AUC = 0.65


In [15]:
pipeline(X_native, y_all, "XGBoostClassifier", "native")

Accuracy = 0.66
F1 = 0.48
ROC_AUC = 0.63


In [16]:
pipeline(X_native, y_all, "CatBoostClassifier", "native")

Accuracy = 0.72
F1 = 0.57
ROC_AUC = 0.71
0:	learn: 0.6650008	total: 668ms	remaining: 1m 24s
1:	learn: 0.6372675	total: 1.19s	remaining: 1m 15s
2:	learn: 0.6106814	total: 1.85s	remaining: 1m 17s
3:	learn: 0.5896977	total: 2.65s	remaining: 1m 22s
4:	learn: 0.5698847	total: 3.35s	remaining: 1m 22s
5:	learn: 0.5479402	total: 3.98s	remaining: 1m 21s
6:	learn: 0.5270331	total: 4.71s	remaining: 1m 21s
7:	learn: 0.5095205	total: 5.34s	remaining: 1m 20s
8:	learn: 0.4882384	total: 6.02s	remaining: 1m 19s
9:	learn: 0.4663995	total: 6.97s	remaining: 1m 22s
10:	learn: 0.4574341	total: 7.72s	remaining: 1m 22s
11:	learn: 0.4405987	total: 8.32s	remaining: 1m 20s
12:	learn: 0.4252011	total: 9.18s	remaining: 1m 21s
13:	learn: 0.4131691	total: 9.75s	remaining: 1m 19s
14:	learn: 0.3974967	total: 10.4s	remaining: 1m 18s
15:	learn: 0.3870358	total: 11.3s	remaining: 1m 18s
16:	learn: 0.3756269	total: 11.8s	remaining: 1m 16s
17:	learn: 0.3647540	total: 12.6s	remaining: 1m 16s
18:	learn: 0.3523289	total: 13.3s

In [17]:
pipeline(X_native, y_all, "RidgeLogistic", "native")

Accuracy = 0.59
F1 = 0.13
ROC_AUC = 0.55
